In [6]:
# 본인의 이름, 학번을 작성하여 실행하세요.
print("정인재")
print("20211927")

정인재
20211927


## 이후 강의 PDF 예제 내용을 작성하여 실행하세요.

In [1]:
class Foo:
    def __getitem__(self, pos):
        return range(0, 30, 10)[pos]

f = Foo()
print(f[1])  
for i in f: 
    print(i)

print(20 in f) 
print(15 in f) 

10
0
10
20
True
False


In [2]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        # '_cards'는 내부적으로 리스트 사용
        self._cards = [Card(rank, suit) for suit in self.suits
                       for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

In [13]:
from random import shuffle
#from frenchdeck import FrenchDeck 은 모듈로 따로 파일을 만들어야 이런식으로 import 가 가능하기 때문에 그냥 기존 클래스를 그대로 작성함
class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        # '_cards'는 내부적으로 리스트 사용
        self._cards = [Card(rank, suit) for suit in self.suits
                       for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

deck = FrenchDeck()
shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

In [5]:
import collections
Card = collections.namedtuple('Card', ['rank', 'suit'])
class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                       for rank in self.ranks]
    def __len__(self):
        return len(self._cards)
    def __getitem__(self, position):
        return self._cards[position]

from random import shuffle
deck = FrenchDeck()

def set_card(deck, position, card):
    deck._cards[position] = card

FrenchDeck.__setitem__ = set_card

shuffle(deck)

print(deck[:5])


[Card(rank='9', suit='clubs'), Card(rank='J', suit='hearts'), Card(rank='6', suit='hearts'), Card(rank='A', suit='diamonds'), Card(rank='J', suit='diamonds')]


In [14]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(collections.abc.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                       for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

    # 1. 항목 할당을 위한 __setitem__ 구현
    def __setitem__(self, position, value): 
        self._cards[position] = value

    # 2. 항목 삭제를 위한 __delitem__ 구현
    def __delitem__(self, position):
        del self._cards[position]

    # 3. 항목 삽입을 위한 insert 구현
    def insert(self, position, value):
        self._cards.insert(position, value)

In [8]:
import abc

class Tombola(abc.ABC):

    @abc.abstractmethod
    def load(self, iterable):
        """Add items from an iterable."""

    @abc.abstractmethod
    def pick(self): 
        """Remove item at random, returning it.
        This method should raise LookupError when the instance is empty.
        """

    def loaded(self):
        """Return True if there's at least 1 item, False otherwise."""
        return bool(self.inspect())

    def inspect(self):
        """Return a sorted tuple with the items currently inside."""
        items = []
        while True: 
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items) 
        return tuple(sorted(items))

In [9]:
import random

class Tombola(abc.ABC):
    @abc.abstractmethod
    def load(self, iterable): pass
    @abc.abstractmethod
    def pick(self): pass
    def loaded(self): return bool(self.inspect())
    def inspect(self):
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(sorted(items))

class BingoCage(Tombola):
    def __init__(self, items):
        self._randomizer = random.SystemRandom() 
        self._items = []
        self.load(items) 

    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)

    def pick(self):
        try:
            return self._items.pop() 
        except IndexError: 
            raise LookupError('pick from empty BingoCage') 

    def __call__(self): 
        return self.pick()

In [10]:
import random

class Tombola(abc.ABC):
    @abc.abstractmethod
    def load(self, iterable): pass
    @abc.abstractmethod
    def pick(self): pass
    def loaded(self): return bool(self.inspect())
    def inspect(self):
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(sorted(items))

class LotteryBlower(Tombola):
    def __init__(self, iterable):
        self._balls = list(iterable) 

    def load(self, iterable):
        self._balls.extend(iterable)

    def pick(self):
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty BingoCage')
        
        return self._balls.pop(position) 

    def loaded(self):
        return bool(self._balls)

    def inspect(self): 
        return tuple(sorted(self._balls))

In [11]:
from random import randrange

class Tombola(abc.ABC):
    @abc.abstractmethod
    def load(self, iterable): pass
    @abc.abstractmethod
    def pick(self): pass
    def loaded(self): return bool(self.inspect())
    def inspect(self):
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(sorted(items))

@Tombola.register 
class TomboList(list): 
    
    def pick(self):
        if self:
            position = randrange(len(self))
            return self.pop(position)
        else:
            raise LookupError('pop from empty TomboList')

    load = list.extend
    
    def loaded(self):
        return bool(self)

    def inspect(self):
        return tuple(sorted(self))